推荐系统入门，使用movielens(文件夹ml-100k)的电影评分数据集。

协同过滤和通过计算余弦相似实现基于内存的CF。

[参考教程](https://www.jianshu.com/p/4df21635d13c)

In [1]:
import pandas as pd
from sklearn import cross_validation as cv
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.metrics import mean_squared_error
from math import sqrt

C:\zMY\Softwares\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
header = ['user_id', 'item_id', 'rating', 'timestamp']
df = pd.read_csv('ml-100k/u.data', sep='\t', names=header)

In [3]:
# 去重后得到一个元祖，分别表示行于列，大小分别位943与1682
n_users = df.user_id.unique().shape[0]
n_items = df.item_id.unique().shape[0]
print('Number of users = ' + str(n_users) + ' | Number of movies = ' + str(n_items))

Number of users = 943 | Number of movies = 1682


In [4]:
# 利用sklearn将数据集分割成测试和训练。此处分割比例0.25。
train_data, test_data = cv.train_test_split(df, test_size=0.25)

## 基于内存的协同过滤
基于内存的协同过滤方法可以分为两个主要部分：用户-产品协同过滤和产品-产品协同过滤。一个用户-产品协同过滤将选取一个特定的用户，基于打分的相似性发现类似于该用户的用户，并推荐那些相似用户喜欢的产品。相比之下，产品-产品协同过滤会选取一个产品，发现喜欢该产品的用户，并找到这些用户或相似的用户还喜欢的其他的产品。输入一个产品，然后输出其他产品作为推荐。

- 用户-产品协同过滤: “喜欢这个东西的人也喜欢……”  
- 产品-产品协同过滤: “像你一样的人也喜欢……”  

在这两种情况下，从整个数据集构建一个用户-产品矩阵。由于你已经将数据拆分到测试集和训练集，那么你将需要创建两个[943 x 1682]矩阵。训练矩阵包含75%的打分，而测试矩阵包含25%的打分。  

In [5]:
# 创建两个用户-物品矩阵，训练和测试
import numpy as np
train_data_matrix = np.zeros((n_users, n_items))
for line in train_data.itertuples():
    train_data_matrix[line[1]-1, line[2]-1] = line[3]
    
test_data_matrix = np.zeros((n_users, n_items))
for line in test_data.itertuples():
    test_data_matrix[line[1]-1, line[2]-1] = line[3]

In [6]:
# 使用sklearn的pairwise_distances计算余弦相似性，同时创建显示性矩阵
user_similarity = pairwise_distances(train_data_matrix, metric='cosine')
item_similarity = pairwise_distances(train_data_matrix.T, metric='cosine')

In [7]:
#下列函数做出预测；评分矩阵，相似度，类型对象
def predict(ratings, similarity, type='user'):
    if type == 'user':
        # 求出用户打分的均值,axis=0表示纵向列取平均值,axis=1表示横向行上求和取均值
        mean_user_rating = ratings.mean(axis=1)
        # 可以百度np.newaxis用法，在此不再赘述
        ratings_diff = (ratings - mean_user_rating[:, np.newaxis])
        pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array(
            [np.abs(similarity).sum(axis=1)]).T
    elif type == 'item':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])
    return pred

In [8]:
# 计算得出相似度预测结果
item_prediction = predict(train_data_matrix, item_similarity, type='item')
user_prediction = predict(train_data_matrix, user_similarity, type='user')

In [9]:
# 利用均方根误差进行评估
def rmse(prediction, ground_truth):
    prediction = prediction[ground_truth.nonzero()].flatten()
    ground_truth = ground_truth[ground_truth.nonzero()].flatten()
    return sqrt(mean_squared_error(prediction, ground_truth))

In [10]:
print('User-based CF RMSE: ' + str(rmse(user_prediction, test_data_matrix)))
print('Item-based CF RMSE: ' + str(rmse(item_prediction, test_data_matrix)))

User-based CF RMSE: 3.118197552200153
Item-based CF RMSE: 3.4453695439142695
